In [5]:
import pandas as pd
import os
from datasets import DatasetDict, Dataset
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, pipeline
import evaluate
import gc
import torch

In [6]:
# Cargar el CSV
csv_path = "/kaggle/input/hwr-test/output_labels.csv"
images_dir = "/kaggle/input/hwr-test/output_lines"

# Leer el CSV
df = pd.read_csv(csv_path)

# Verificar que las imágenes existen
def check_image_exists(row):
    return os.path.exists(os.path.join(images_dir, row['filename']))

df = df[df.apply(check_image_exists, axis=1)]


# Dividir en train/test (ajusta según necesites)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Crear datasets HuggingFace
def create_dataset(dataframe):
    dataset = Dataset.from_dict({
        'image': [Image.open(os.path.join(images_dir, fname)).convert('RGB') for fname in dataframe['filename']],
        'text': list(dataframe['label'])
    })
    return dataset

dataset = DatasetDict({
    'train': create_dataset(train_df),
    'test': create_dataset(test_df)
})

In [7]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

def prepare_examples(batch):
    images = batch['image']
    texts = batch['text']
    
    # Procesar imágenes
    pixel_values = processor(images, return_tensors="pt").pixel_values
    
    # Procesar textos
    labels = processor.tokenizer(texts, padding="max_length", max_length=64).input_ids
    
    # Reemplazar padding token id por -100 para ignorar en loss
    labels = [[label if label != processor.tokenizer.pad_token_id else -100 for label in labels_example] for labels_example in labels]
    
    batch['pixel_values'] = pixel_values
    batch['labels'] = labels
    
    return batch

# Aplicar el preprocesamiento
train_dataset = dataset['train'].map(prepare_examples, batched=True, batch_size=8)
eval_dataset = dataset['test'].map(prepare_examples, batched=True, batch_size=8)

# Configurar formato para PyTorch
train_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])
eval_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

# Configurar parámetros del modelo
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# Configuración para generación
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    # Agrega esto a tus compute_metrics
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"cer": cer, "wer": wer}
    
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-trained",
    per_device_train_batch_size=4,  # Aumentar si tu GPU lo permite
    per_device_eval_batch_size=4,
    num_train_epochs=10,  # Más épocas
    fp16=True,
    learning_rate=3e-5,  # Tasa de aprendizaje más baja
    eval_strategy="steps",
    eval_steps=200,  # Evaluar con menos frecuencia
    save_steps=500,
    logging_steps=50,
    warmup_steps=100,  # Añadir warmup
    weight_decay=0.01,  # Regularización
    save_total_limit=2,
    predict_with_generate=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/models/trocr/processing_trocr.py:152: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/tmp/ipykernel_267/1178572990.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
gc.collect()
torch.cuda.empty_cache()

# Iniciar entrenamiento
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=50, training_loss=2.497156677246094, metrics={'train_runtime': 56.6357, 'train_samples_per_second': 7.063, 'train_steps_per_second': 0.883, 'total_flos': 3.539581802643456e+17, 'train_loss': 2.497156677246094, 'epoch': 10.0})

In [11]:
# Guardar el modelo entrenado
trainer.save_model("trocr-trained-custom")
processor.save_pretrained("trocr-trained-custom")

[]

In [14]:
images_dir = "/kaggle/input/image-test/"

# Cargar el modelo entrenado
ocr = pipeline("image-to-text", model="./trocr-trained-custom")

# Probar con una imagen de test
test_image = Image.open(os.path.join(images_dir, "test_image.png")).convert("RGB")
prediction = ocr(test_image)
print(f"Predicción: {prediction[0]['generated_text']}")

Device set to use cuda:0


Predicción: Enque Francisco
